In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 112.0.5615
Get LATEST chromedriver version for 112.0.5615 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/112.0.5615.49/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\liz14\.wdm\drivers\chromedriver\win32\112.0.5615.49]


In [2]:
url = "https://www.imdb.com/list/ls006444168/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [48]:
# Webscrape imdb movies from books list. 
movies_list = soup.find_all('div', class_='lister-list')

# Pull movie title, year released, movie duration, genre, movie rating ("certificate"), star rating, gross amount.

title = movies_list[0].find('h3').find('a').text

year = movies_list[0].find('span',class_='lister-item-year').text

movie_length = movies_list[0].find('span',class_='runtime').text

genre = movies_list[0].find('span',class_='genre').text

# javascript for certificate: # <span class="certificate">R</span>
certificate = movies_list[0].find('span',class_='certificate').text

# javascript for rating: <span class="ipl-rating-star__rating">7.5</span>
stars = movies_list[0].find('span',class_='ipl-rating-star__rating').text

# javascript for movie gross:
# <span class="text-muted">Gross:</span>
# <span name="nv" data-value="18,335,230">$18.34M</span>

# chatGPT   
gross = movies_list[0].find('span', string='Gross:')
gross_value = gross.find_next_sibling('span')['data-value']

#********************Discard later?
# # find the <p> tag with class 'text_muted text-small'
##p_tag = soup.find('p', class_='text-muted text-small')
#p_tag = soup.find('span', class_='text-muted').text
#p_tag = soup.find('span', name_='nv')


# # find the <span> tag with name 'nv' inside the <p> tag
#span_tag = p_tag.find('span', name='nv')

# # extract the text content of the <span> tag.
# text = span_tag.text.strip()

In [ ]:
movies_list

In [55]:
# Pull movie title, year released, movie duration, genre, movie rating ("certificate"), star rating, gross amount.

# Create dictionary of lists to hold....

movies_fr_books_dict = {}
movie_title =[]
year = []
movie_length =[]
movie_genre=[]
movie_rating=[]
star_rating=[]
gross_amount=[]

def get_movietitle():
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    movies_list = soup.find_all('div', class_='lister-item-content')
    
    for movie in movies_list:
        title = movie.find('h3').find('a').text
        
        movie_year = movie.find('span', class_='lister-item-year').text
        
        movie_len = movie.find('span', class_='runtime').text
        
        genre = movie.find('span', class_='genre').text.strip()
    
        certificate = movie.find('span', class_='certificate')
        
        stars = movie.find('span', class_='ipl-rating-star__rating').text
        
        gross = movies_list[0].find('span', string='Gross:')
        gross_value = gross.find_next_sibling('span')['data-value']

        # recording values
        movie_title.append(title)
        year.append(movie_year)
        movie_length.append(movie_len)
        movie_genre.append(genre)        
        movie_rating.append(certificate)
        star_rating.append(stars)
        gross_amount.append(gross_value)

get_movietitle()

In [56]:
movies_fr_books_dict['movie_title'] = movie_title
movies_fr_books_dict['year'] = year
movies_fr_books_dict['movie_length'] = movie_length
movies_fr_books_dict['movie_genre'] = movie_genre
movies_fr_books_dict['movie_rating'] = movie_rating
movies_fr_books_dict['star_rating'] = star_rating
movies_fr_books_dict['gross_amount'] = gross_value

In [64]:
import pandas as pd
moviesfrombooks_data = pd.DataFrame(movies_fr_books_dict)
moviesfrombooks_data.head(5)

,movie_title,year,movie_length,movie_genre,movie_rating,star_rating,gross_amount
0,127 Hours,(2010),94 min,"Biography, Drama",[R],7.5,"18,335,230"
1,100 Rifles,(1969),110 min,"Adventure, Drama, War",[R],6,"18,335,230"
2,The Hunger Games,(2012),142 min,"Action, Adventure, Sci-Fi",[PG-13],7.2,"18,335,230"
3,The Hunger Games: Catching Fire,(2013),146 min,"Action, Adventure, Sci-Fi",[PG-13],7.5,"18,335,230"
4,A Beautiful Mind,(2001),135 min,"Biography, Drama",[PG-13],8.2,"18,335,230"


In [58]:
# Display random sample of 10 rows from dataframe, moviesfrombooks_data

moviesfrombooks_data.sample(n=10)

,movie_title,year,movie_length,movie_genre,movie_rating,star_rating,gross_amount
99,Revolutionary Road,(2008),119 min,"Drama, Romance",[R],7.3,"18,335,230"
78,The Lincoln Lawyer,(2011),118 min,"Crime, Drama, Mystery",[R],7.3,"18,335,230"
87,The Runaways,(2010),106 min,"Biography, Drama, Music",[R],6.5,"18,335,230"
14,Message in a Bottle,(1999),126 min,"Drama, Romance",[PG-13],6.2,"18,335,230"
98,Inkheart,(2008),106 min,"Adventure, Family, Fantasy",[PG],6.1,"18,335,230"
97,Confessions of a Shopaholic,(2009),104 min,"Comedy, Romance",[PG],5.8,"18,335,230"
22,The Girl with the Dragon Tattoo,(2011),158 min,"Crime, Drama, Mystery",[R],7.8,"18,335,230"
11,About a Boy,(2002),101 min,"Comedy, Drama, Romance",[PG-13],7.1,"18,335,230"
64,The Lovely Bones,(2009),135 min,"Drama, Fantasy, Thriller",[PG-13],6.6,"18,335,230"
25,The Golden Compass,(2007),113 min,"Adventure, Family, Fantasy",[PG-13],6.1,"18,335,230"


In [63]:
# Save/export dataframe, moviesfrombooks_data, to csv file.

moviesfrombooks_data.to_csv('Resources/imdb_movies_from_books.csv', index=False)

In [66]:
browser.quit()